# Comment construire un moteur de recherche ?

# Pré-requis : Importation

In [185]:
import os
import re


#import pour stopwords
from collections import Counter
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop=stopwords.words('english')

#time
from time import time

#serialisation
import pickle

#affichage set
import itertools

#maths
import math

#operation matrice
import operator

#modele espace vectoriel
import numpy as np
from scipy.sparse import *
from scipy import *

#mapReduce
from multiprocessing import Pool
from functools import reduce

# Stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download("punkt")
ps = PorterStemmer()

# Lemmatization
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. INDEXATION

### 1.a. Lire les données  : fonction createCorpus()
retourne une liste de dictionnaires

list ([auteur/article.txt]=texte, [auteur/article.txt]=texte, [auteur/article.txt]=texte, ... )

In [23]:
def createCorpus():
    #création du corpus : Auteur, Article, Text
    listAuthor = os.listdir("data")
    listText = {}
    for author in listAuthor :
        listArticle = os.listdir("data/"+author)
        for article in listArticle :
            with open("data/"+author+"/"+article,"r") as f :
                listText[author+"/"+article] = f.read()
    return listText 

In [24]:
listText=createCorpus()
listItems = list(listText.items())
print("Affichage du premier élement de la liste : \n")
print("nom de l'article : ", listItems[0][0],"\n")
print("texte : ", listItems[0][1])

Affichage du premier élement de la liste : 

nom de l'article :  LynneO'Donnell/204874newsML.txt 

texte :  Some Chinese traders are moving to take advantage of a sharp rally in world copper prices by selling the red metal back to the market, Far East metal industry sources said on Wednesday.
A rise in copper stocks held in London Metal Exchange (LME) warehouses in Singapore was attributed in part by some trading sources to Chinese selling.
"With such high LME prices, I believe some (copper) will be brought out of China," a Singapore trader said.  
He said stocks in Singapore rose 5,000 to 6,000 tonnes late last week -- after hovering at extremely low levels for the past couple of months -- as Chinese buyers sold material recently taken from LME warehouses elsewhere.
Asian traders have said China is adding substantial tonnages -- up to 300,000 tonnes -- to its central stockpile, with a third of that amount now passing through customs in Shanghai.
These reports have helped fuel a rally 

### 1.b. Création de l'index inversé : fonction createIndex()
Création de l'index de base (non distribué, sans les stopwords)
la fonction de création d'index appelle la fonction createCorpus

retourne une liste de dictionnaires de dictionnaires : 


index ( mot : { titre : {nbMots, occurrence, position }, titre : { nbMots, occurrence, position }, ... },
mot { titre: {nbMots, occurrence, position}, ... }, ... )

In [25]:
def createIndex():
    listText=createCorpus()
    #création du vocabulaire et de l'index en même temps
    indexBis = {}
    listItems = list(listText.items())
    #parcourir tous les textes présents
    for i in range(0,len(listText)): 
        #normalisation : récupération du texte sans les stopwords
        text = re.findall(r'[a-z]+', listItems[i][1].lower())
        text = [word for word in text if word not in stop]
        size = len(text) #taille du texte
        dicoPos = {} #dictionnaire temporaire pour cposition mot
        dicoOcc = {} #dictionnaire temporaire pour occurence mot
        #parcourir chaque mot du texte i
        for j in range(0, len(text)) :
            #si le mot n'est pas déjà présent dans l'index
            if (text[j] not in dicoPos.keys()) :
                dicoPos[text[j]] = [j]  #position du mot
                dicoOcc[text[j]] = 1 #occurence égale à 1
            else :
                dicoPos[text[j]].append(j) #ajoute la nouvelle position du mot déjà présent
                dicoOcc[text[j]] += 1 # + 1 à l'occurence du mot
        doc = listItems[i][0]
        #pour tous les mots du dicoTmp
        for word in dicoPos.keys():
            #si le mot n'est pas déjà dans l'index on ajoute liste vide
            if word not in indexBis.keys():
                indexBis[word] = []
            #on ajoute pour le mot, les articles(=doc), la taille, les occurences et les positions de chaque texte
            indexBis[word].append([doc, size, dicoOcc[word], dicoPos[word]])

    #creation de l'index final    (index inversé)     
    index={} 
    for word in indexBis.keys():
        index[word]={}
        for doc in indexBis[word]:
            index[word][doc[0]]={}
            index[word][doc[0]]["nbMots"]=doc[1]
            index[word][doc[0]]["occ"]=doc[2]
            index[word][doc[0]]["pos"]=doc[3]
    return index

Temps d'éxécution de la méthode

In [26]:
d = time()
monIndex=createIndex()
f = time()
print("temps d\'execution de createIndex() : ", f-d)


temps d'execution de createIndex() :  4.100175619125366


Nombre de mots dans l'index

In [27]:
print("Nombre de mots différents dans l'index : ", len(monIndex))

Nombre de mots différents dans l'index :  27361


Afficher les 10 premiers articles dans lesquels figurent le mot "merchandises"

In [197]:
dict(itertools.islice(monIndex["merchandise"].items(), 10))

{'AaronPressman/156814newsML.txt': {'nbMots': 349, 'occ': 1, 'pos': [51]},
 'BradDorfman/199931newsML.txt': {'nbMots': 333, 'occ': 1, 'pos': [34]},
 'BradDorfman/276360newsML.txt': {'nbMots': 413, 'occ': 1, 'pos': [212]},
 'BradDorfman/331819newsML.txt': {'nbMots': 308, 'occ': 1, 'pos': [198]},
 'BradDorfman/346831newsML.txt': {'nbMots': 297, 'occ': 1, 'pos': [72]},
 'JohnMastrini/159754newsML.txt': {'nbMots': 310,
  'occ': 3,
  'pos': [76, 91, 129]},
 'MartinWolk/183836newsML.txt': {'nbMots': 222, 'occ': 2, 'pos': [87, 116]},
 'MichaelConnor/372558newsML.txt': {'nbMots': 347, 'occ': 1, 'pos': [192]},
 'RobinSidel/113551newsML.txt': {'nbMots': 292,
  'occ': 3,
  'pos': [146, 210, 217]},
 'SamuelPerry/10781newsML.txt': {'nbMots': 359, 'occ': 1, 'pos': [223]}}

### 1.c. Création de l'index inversé : fonction createIndexMapReduce() --> Utilisation MapReduce et Stemming

-Parallélise le calcul (Map/Reduce)

-Ajout de stemming

Besoin d'une nouvelle fonction createCorpusBis() : création d'un corpus qui retourne une liste de dictionnaires ( dic{titre,text} )

In [29]:
def createCorpusBis():
    #création du corpus : Auteur, Article, Text
    #liste de dictionnaires
    listAuthor = os.listdir("data")
    listText = []
    i=0
    for author in listAuthor :
        listArticle = os.listdir("data/"+author)
        for article in listArticle :
            with open("data/"+author+"/"+article,"r") as f :
                dicTmp={}
                dicTmp["titre"]=author+"/"+article
                dicTmp["id"]=i
                dicTmp["texte"]=f.read()
                listText.append(dicTmp)
                i=i+1
    return listText

In [30]:
listCorpus=createCorpusBis()
print("Affichage du premier élement de la liste : \n")
print("nom de l'article : ", listCorpus[0]["titre"],"\n")
print("id : ", listCorpus[0]["id"],"\n")
print("texte : ", listCorpus[0]["texte"])


Affichage du premier élement de la liste : 

nom de l'article :  LynneO'Donnell/204874newsML.txt 

id :  0 

texte :  Some Chinese traders are moving to take advantage of a sharp rally in world copper prices by selling the red metal back to the market, Far East metal industry sources said on Wednesday.
A rise in copper stocks held in London Metal Exchange (LME) warehouses in Singapore was attributed in part by some trading sources to Chinese selling.
"With such high LME prices, I believe some (copper) will be brought out of China," a Singapore trader said.  
He said stocks in Singapore rose 5,000 to 6,000 tonnes late last week -- after hovering at extremely low levels for the past couple of months -- as Chinese buyers sold material recently taken from LME warehouses elsewhere.
Asian traders have said China is adding substantial tonnages -- up to 300,000 tonnes -- to its central stockpile, with a third of that amount now passing through customs in Shanghai.
These reports have helped fue

Définition de la fonction mapIndex (phase de Map)
    On retourne un dictionnaire d'un dictionnaire d'un dictionnaire d'un dictionnaire pour un article donné :
    

index : {mot : {titre : {idArticle: { positionMot, nbMots, occurence }}, titre: {idArticle: {  positionMot, nbMots, occurence }}}, ... ,
 mot : {titre : {idArticle: { positionMot, nbMots, occurence }}, titre: {idArticle: {  positionMot, nbMots, occurence }}}, ... ,}


titre : auteur/article.txt
positionMot : liste des positions du mot dans le texte
nbMots : nombre de mots présents dans le texte
occurence : nombre de fois où le mot apparait dans le texte

On créer ici directement l'index inversé

In [31]:
def mapIndex(article):
    text = re.findall(r'[a-z]+', article["texte"].lower())
    text = [word for word in text if word not in stop]
    text = [ps.stem(word) for word in text] ### STEMMING
    size = len(text) #taille du texte


    indexRes = {} #initialisation index de sortie (dictionnaire)
    #parcourir chaque mot du texte i
    for i in range(0, len(text)) :
        word=text[i]
        #si le mot n'est pas déjà présent dans l'index
        if (word not in indexRes.keys()) :
            indexRes[word]={} ## creation du dictionnaire
            indexRes[word][article["titre"]]={} ##creation du second dictionnaire avec pour clé le titre (auteur/article)
            indexRes[word][article["titre"]]["pos"]=[]# initialisation de la valeur position
            indexRes[word][article["titre"]]["nbMots"]=size #indique le nbMots dans le texte
            indexRes[word][article["titre"]]["idArticle"]=article["id"]
        indexRes[word][article["titre"]]["pos"].append(i) #ajoute la nouvelle position
    
    #boucle pour pour valoriser l'occurence du mot
    for word in indexRes.keys():
        indexRes[word][article["titre"]]["occ"]=len( indexRes[word][article["titre"]]["pos"])
    return indexRes

Définition de la fonction reduceIndex (phase de Reduce)


Prend en entrée deux index et retourne qu'un seul index en combinant les mots des deux index.

In [32]:
def reduceIndex(index1,index2):
    for word in index2.keys(): #pour chaque mot de l'index 2
        if word not in index1.keys() : #si le mot n'est pas présent dans l'index 1 
            index1[word]={} #on rajoute ce mot
        info2=list(index2[word].keys())[0] #on prend toutes les informations concernant le mot de l'index 2
        index1[word][info2]=index2[word][info2] #on ajoute ces information dans l'index 1
    return index1 #on retourne l'index1 qui combine maintenant les deux index  

Défintion de la fonction createIndexMapRed()

Boucle pool en reprenant les fonctions map et reduce
le map partage le temps de calcul : parallélisation du temps pour créer les index de chaque article
le reduce vient combiner les infos de chaque index créé en réunissant les mots communs

In [33]:
 def createIndexMapRed():
    listText=createCorpusBis() #creation du corpus (liste de dictionnaire)
    with Pool() as pool:
        #Map
        dMap = time()
        listIndex=pool.map(mapIndex,listText) #appel fonction map sur listText
        fMap = time()
        print('temps d\'execution du Map : ', fMap-dMap)
        
        #Reduce
        dRed = time()
        indexRes=reduce(reduceIndex,listIndex) #reduce sur listIndex en sortie du map
        fRed = time()
        print('temps d\'execution du Reduce : ', fRed-dRed)
    return indexRes

In [34]:
d = time()
monIndexMap=createIndexMapRed()
f = time()
print('temps d\'execution total de createIndexMapReduce', f-d)

temps d'execution du Map :  5.093148708343506
temps d'execution du Reduce :  0.4122588634490967
temps d'execution total de createIndexMapReduce 5.798810005187988


Nombre de mots de l'index : avec le Stemming moins de mots que l'index simple

In [38]:
print("Nombre de mots différents dans l'index avec stemming : ", len(monIndexMap))
print("Nombre de mots différents dans l'index : ", len(monIndex))

Nombre de mots différents dans l'index avec stemming :  18456
Nombre de mots différents dans l'index :  27361


monIndexMap est une liste de dictionnaire de dictionnaire de dictionnaire : 


index ( mot : { titre : {idArticle, nbMots, occurence, position }, titre : {idArticle, nbMots, occurence, position }, ... },
mot { titre: {idArticle, nbMots, occurence, position}, ... }, ... )

Afficher les 10 premiers articles dans lesquels figurent le mot "merchandises"

In [36]:
dict(itertools.islice(monIndexMap["merchandis"].items(), 10))

{'AaronPressman/156814newsML.txt': {'idArticle': 305,
  'nbMots': 349,
  'occ': 1,
  'pos': [51]},
 'BradDorfman/199931newsML.txt': {'idArticle': 1207,
  'nbMots': 333,
  'occ': 1,
  'pos': [34]},
 'BradDorfman/276360newsML.txt': {'idArticle': 1217,
  'nbMots': 413,
  'occ': 1,
  'pos': [212]},
 'BradDorfman/331819newsML.txt': {'idArticle': 1214,
  'nbMots': 308,
  'occ': 2,
  'pos': [198, 227]},
 'BradDorfman/346831newsML.txt': {'idArticle': 1222,
  'nbMots': 297,
  'occ': 1,
  'pos': [72]},
 'EricAuchard/149671newsML.txt': {'idArticle': 381,
  'nbMots': 467,
  'occ': 1,
  'pos': [207]},
 'KeithWeir/275481newsML.txt': {'idArticle': 1639,
  'nbMots': 539,
  'occ': 1,
  'pos': [107]},
 'MichaelConnor/372558newsML.txt': {'idArticle': 1144,
  'nbMots': 347,
  'occ': 1,
  'pos': [192]},
 'PatriciaCommins/143591newsML.txt': {'idArticle': 536,
  'nbMots': 336,
  'occ': 1,
  'pos': [197]},
 'RobinSidel/219316newsML.txt': {'idArticle': 1701,
  'nbMots': 220,
  'occ': 1,
  'pos': [198]}}

### 1.d. Serialization

In [39]:
#sauvegarde de mon index
pickle.dump(monIndexMap,open("save.p","wb"))
#importer mon index depuis le fichier save.p
indexImport = pickle.load( open( "save.p", "rb" ) )

In [40]:
monIndexMap=indexImport

# 2. REQUETES

### 2.a.  Répondre à une requête en un  : dans quel article un mot apparait-il ?

Fonction pour rechercher un mot

In [41]:
def Research1Word(index,word):
    word2=re.findall(r'[a-z]+',word.lower()) 
    word2 = [ps.stem(word) for word in word2] ### STEMMING
    try:
        return set((index[word2[0]].keys())) #retourne les infos du mot
    except:
        print("Le mot " + word +" n'existe pas dans l'index \n")
        return set()

Temps d'exécution pour la recherche d'un mot

In [203]:
d=time()
Research1Word(monIndexMap,'merchandises')
f=time()
print("temps d\'execution de Research1Word() : ", f-d)

temps d'execution de Research1Word() :  9.775161743164062e-05


Fonction pour afficher la recherche à un mot

In [42]:
def PrintResearch1(index):
    dicDoc={}
    cont="oui"
    while(cont=="oui"):
        motRech=input("Indiquez le mot que vous souhaitez rechercher : \n")
        listDoc=Research1Word(index,motRech)
        print("nombre de documents = %d \n" %len(listDoc))
        if len(listDoc)>0 : 
            print("Voici les 10 premiers éléments : \n ", set(itertools.islice(listDoc,10)))
        cont=input("Souhaitez vous rechercher un autre mot ? (oui/non) \n").lower()
    print("Fin de recherche à 1 mot")

Tester la fonction : (avec le mode affichage)

In [43]:
#tester la fonction
PrintResearch1(monIndexMap)

Indiquez le mot que vous souhaitez rechercher : 
the
Le mot the n'existe pas dans l'index 

nombre de documents = 0 

Souhaitez vous rechercher un autre mot ? (oui/non) 
oui
Indiquez le mot que vous souhaitez rechercher : 
merChandisE
nombre de documents = 23 

Voici les 10 premiers éléments : 
  {'MarkBendeich/358701newsML.txt', 'JohnMastrini/18529newsML.txt', 'BradDorfman/346831newsML.txt', 'PatriciaCommins/143591newsML.txt', 'JohnMastrini/123158newsML.txt', 'SamuelPerry/10781newsML.txt', 'TheresePoletti/157001newsML.txt', 'BradDorfman/331819newsML.txt', 'AaronPressman/156814newsML.txt', 'RobinSidel/113551newsML.txt'}
Souhaitez vous rechercher un autre mot ? (oui/non) 
non
Fin de recherche à 1 mot


### 2.b. Répondre à une requête en texte libre : quels articles contient au moins un mot d'une liste de mots donnée ?

Fonction pour rechercher une liste de mots

In [44]:
def ResearchWord(index,listWord):
    allDoc=set()
    for word in listWord:
        #utilisation de la fonction précédente pour rechercher un mot  
        docWord=Research1Word(index,word)  
        for doc in docWord:
            allDoc.add(doc)
    return allDoc

Temps d'exécution pour la recherche de texte libre

In [204]:
d=time()
ResearchWord(monIndexMap,['merchandise','traders','deficit'])
f=time()
print("temps d\'execution de ResearchWord() : ", f-d)

temps d'execution de ResearchWord() :  0.0003094673156738281


Fonction pour afficher la recherche

In [45]:
def PrintResearch(index):
    cont="oui"
    wordUse=[]
    while(cont=="oui"):
        motRech=re.findall(r'[a-z]+',input("Indiquez les mots (en texte libre) que vous souhaitez rechercher : \n").lower())
        #on splite les mots
        for mot in motRech:
            wordUse.append(mot)
        if (len(wordUse)>0):
            listDoc=ResearchWord(index,wordUse)
            print("nombre de documents = %d \n" %len(listDoc))
            if len(listDoc)>0 : 
                print("Voici les 10 premiers éléments : \n ", set(itertools.islice(listDoc,10)))
        cont=input("Souhaitez vous rechercher d'autres mots en texte libre ? (oui/non) \n").lower()
    print("Fin de recherche en texte libre")

Tester la fonction : (avec le mode affichage)

In [46]:
#test the puis MercHANdisers traDer
PrintResearch(monIndexMap)

Indiquez les mots (en texte libre) que vous souhaitez rechercher : 
the
Le mot the n'existe pas dans l'index 

nombre de documents = 0 

Souhaitez vous rechercher d'autres mots en texte libre ? (oui/non) 
oui
Indiquez les mots (en texte libre) que vous souhaitez rechercher : 
MercHandiserS Trader
Le mot the n'existe pas dans l'index 

nombre de documents = 197 

Voici les 10 premiers éléments : 
  {'AlexanderSmith/240608newsML.txt', 'LydiaZajc/172241newsML.txt', 'RobinSidel/186416newsML.txt', 'GrahamEarnshaw/167951newsML.txt', 'JoeOrtiz/343304newsML.txt', 'MarkBendeich/402412newsML.txt', 'MarcelMichelson/150780newsML.txt', 'NickLouth/13825newsML.txt', 'BradDorfman/344162newsML.txt', 'EricAuchard/133874newsML.txt'}
Souhaitez vous rechercher d'autres mots en texte libre ? (oui/non) 
non
Fin de recherche en texte libre


### 2.c. Répondre à une requête d'expression : Quels articles contient une phrase donnée en respectant l'ordre des mots ?

Définitions de quelques fonctions (en rapport avec le stemming)
Necessaire de faire appel au stemming lors d'une recherche direct dans l'index

In [50]:
#rechercher la ou les positions d'un mot à partir d'un titre connu
def searchPos(index,word,titre):
    word=ps.stem(word)
    pos=index[word][titre]["pos"]
    return pos


#rechercher l'occurence d'un mot à partir d'un titre connu
def searchOcc(index,word,titre):
    word=ps.stem(word)
    occ=index[word][titre]["occ"]
    return occ

#rechercher le nombre de mots d'un mot à partir d'un titre connu
def searchNbMots(index,word,titre):
    word=ps.stem(word)
    nbMots=index[word][titre]["nbMots"]
    return nbMots

#rechercher l'id de l'article d'un mot à partir d'un titre connu
def searchIdArticle(index,word,titre):
    word=ps.stem(word)
    idArticle=index[word][titre]["idArticle"]
    return idArticle

Définition de la fonction qui recherche une phrase dans l'index
L'ordre des mots est important (utilisation de la fonction searchPos)

In [51]:
def ResearchSeqWord(index,phrase):
    allDoc=set()
    #normalisation de la phrase et split en liste de mots
    l = re.findall(r'[a-z]+', phrase.lower())
    listWord = [word for word in l if word not in stop] #on enleve les stopwords pour ne pas les chercher inutilement
    #pas besoin de stemming ici, puisqu'il est present dans Research1Word appele plus loin dans la fonction
    if len(listWord)<=1:
        listDoc=Research1Word(index,listWord[0])
    else:
        docWord1=ResearchWord(index,[listWord[0]])
        docWord2=ResearchWord(index,[listWord[1]])
        #recuperation des articles communs aux deux premiers mots
        #comme on cherche les articles ayant une phrase commune, les deux premiers mots suffisents
        allDoc=docWord1.intersection(docWord2)

        listDoc=[]
        #pour chaque article trouvé
        for doc in allDoc:
            posOk=True
            j=0
            #tant que la liste des mots n'a pas été parcouru et si pasOk est true
            while (j<len(listWord)-1) and posOk:
                word1=listWord[j] #mot1
                word2=listWord[j+1] #mot2

                docWord1=Research1Word(index,word1) #documents correspondants au mot1
                docWord2=Research1Word(index,word2) #documents correspondants au mot2
                
                allDocBis=docWord1.intersection(docWord2) #documents communs aux mot1 et mot2
                if doc in allDocBis: #si le document sur lequel on travail est présent dans la liste des documents communs
                   #necessaire de faire du stemming en amont pour pouvoir rechercher directi

                    pos1=[i+1 for i in searchPos(index,word1,doc)] # on ajoute 1 a la position du mot 1 --> position mot1+1
                    pos2=searchPos(index,word2,doc) #position mot2
                    #on cherche si pos1=pos2
                    inter=set(pos1).intersection(pos2) #intersection des deux mots
                    #si inter est non vide alors les deux mots se suivent dans le texte
                    if (len(inter)>0):
                        posOk=True
                        j=j+1
                    else:
                        posOk=False  #pour arreter la boucle
                else: #si le document sur lequel on travail n'est pas présent dans la liste des documents communs
                    posOk=False #pour arreter la boucle
                    
            if j==len(listWord)-1 and posOk: #si le compteur j est égal à taille de liste de mots 
                #on est allé jusqu'à la fin de la phrase. Donc elle est présente dans l'article doc
                listDoc.append(doc) #ajoute à la liste des documents
    return listDoc

In [52]:
def PrintResearchSeq(index):
    cont="oui"
    wordUse=[]
    while(cont=="oui"):
        motRech=input("Indiquez la phrase que vous souhaitez rechercher : \n")
        motRechBis=re.findall(r'[a-z]+', motRech.lower())
        listWord = [word for word in motRechBis if word not in stop] 
        #on splite les mots
        for mot in listWord:
            wordUse.append(mot)
        if (len(wordUse)>0):
            listDoc=ResearchSeqWord(index,motRech)
            print("nombre de documents = %d \n" %len(listDoc))
            if len(listDoc)>0:
                print("Voici les 10 premiers éléments : \n ", set(itertools.islice(listDoc,10)))
        cont=input("Souhaitez vous rechercher une autre phrase ? (oui/non) \n").lower()
    print("Fin de recherche en phrase")

Test de la fonction sur différentes phrases
On vérifie en même temps que la recherche n'est pas sensible à la casse

In [53]:
# pas sensible à la casse avec une phrase : The MercHaNdise Trade DEficit
PrintResearchSeq(monIndexMap)

Indiquez la phrase que vous souhaitez rechercher : 
couc the
Le mot couc n'existe pas dans l'index 

nombre de documents = 0 

Souhaitez vous rechercher une autre phrase ? (oui/non) 
oui
Indiquez la phrase que vous souhaitez rechercher : 
The MercHaNdise Trade DEficit
nombre de documents = 5 

Voici les 10 premiers éléments : 
  {'JohnMastrini/18529newsML.txt', 'JohnMastrini/159754newsML.txt', 'JohnMastrini/13806newsML.txt', 'JohnMastrini/123158newsML.txt', 'JohnMastrini/18297newsML.txt'}
Souhaitez vous rechercher une autre phrase ? (oui/non) 
non
Fin de recherche en phrase


In [54]:
#phrase : Lloyd's of London Reconstruction and Renewal plan
PrintResearchSeq(monIndexMap)

Indiquez la phrase que vous souhaitez rechercher : 
Lloyd's of London Reconstruction and Renewal plan
nombre de documents = 1 

Voici les 10 premiers éléments : 
  {'SimonCowell/376405newsML.txt'}
Souhaitez vous rechercher une autre phrase ? (oui/non) 
non
Fin de recherche en phrase


In [55]:
##fonctionne aussi pour un seul mot --> day
PrintResearchSeq(monIndexMap)

Indiquez la phrase que vous souhaitez rechercher : 
day
nombre de documents = 659 

Voici les 10 premiers éléments : 
  {'AlanCrosby/21296newsML.txt', 'KouroshKarimkhany/129309newsML.txt', 'SimonCowell/406879newsML.txt', 'JaneMacartney/279102newsML.txt', 'TanEeLyn/462149newsML.txt', 'MatthewBunce/236509newsML.txt', 'BradDorfman/196808newsML.txt', 'EricAuchard/191274newsML.txt', 'BenjaminKangLim/201374newsML.txt', 'JaneMacartney/272173newsML.txt'}
Souhaitez vous rechercher une autre phrase ? (oui/non) 
non
Fin de recherche en phrase


Temps d'exécution pour la recherche d'une phrase

In [206]:
d=time()
ResearchSeqWord(monIndexMap,"The MercHaNdise Trade DEficit")
f=time()
print("temps d\'execution de ResearchSeqWord() : ", f-d)

temps d'execution de ResearchSeqWord() :  0.0021505355834960938


## 3. RANKING : 

### 3.a. Ordonner les résultats d'une requête simple par nombre d'occurences

In [56]:
def RankingOrderOcc(index,word):
    articles=Research1Word(index,word) #utilisation de la fonction rechercher un mot
    #si le mot en paramètre n'existe pas, except de Research1Word
    wordBis=re.findall(r'[a-z]+', word.lower())
    #wordBis = [ps.stem(word) for word in wordBis] ### STEMMING
    dico={}
    for doc in articles: #pour chaque article
        #doc[1] correspond a l'occurence du mot de l'index
        dico[doc]=searchOcc(index,wordBis[0],doc) #ajout dans le dico de cle 'doc' l'occurence du mot word
    dicoSorted = sorted(dico.items(), reverse=True, key=lambda x:x[1]) #trier le dico pour ordonner resultat
    return dicoSorted

Test de la fonction : 

In [57]:
RankingOrderOcc(monIndexMap,'merchandise')

[('RobinSidel/113551newsML.txt', 4),
 ('JohnMastrini/159754newsML.txt', 3),
 ('TheresePoletti/157001newsML.txt', 2),
 ('BradDorfman/331819newsML.txt', 2),
 ('MartinWolk/183836newsML.txt', 2),
 ('MarkBendeich/358701newsML.txt', 1),
 ('JohnMastrini/18529newsML.txt', 1),
 ('PatriciaCommins/143591newsML.txt', 1),
 ('JohnMastrini/123158newsML.txt', 1),
 ('AaronPressman/156814newsML.txt', 1),
 ('BradDorfman/346831newsML.txt', 1),
 ('SamuelPerry/10781newsML.txt', 1),
 ('BradDorfman/276360newsML.txt', 1),
 ('JohnMastrini/18297newsML.txt', 1),
 ('RobinSidel/220676newsML.txt', 1),
 ('MichaelConnor/372558newsML.txt', 1),
 ('BradDorfman/199931newsML.txt', 1),
 ('MarkBendeich/358695newsML.txt', 1),
 ('JohnMastrini/13806newsML.txt', 1),
 ('RobinSidel/219316newsML.txt', 1),
 ('EricAuchard/149671newsML.txt', 1),
 ('KeithWeir/275481newsML.txt', 1),
 ('JaneMacartney/106841newsML.txt', 1)]

dans l'article 'RobinSidel/113551newsML.txt' il y a 4 mots dont la racine est merchandis
Si l'on regarde plus précisemment dans ce texte : 

In [58]:
monIndex["merchandise"]["RobinSidel/113551newsML.txt"]

{'nbMots': 292, 'occ': 3, 'pos': [146, 210, 217]}

In [587]:
monIndex["merchandisers"]["RobinSidel/113551newsML.txt"]

{'nbMots': 292, 'occ': 1, 'pos': [194]}

Dans cet article, il y a en réalité 3 mots "merchandise" et 1 mot "merchandisers". Donc avec le stemming on arrive à combiner les deux sous la racine "merchandis" et à en compter 4.
Sans le stemming, on au deux mots différents avec chacun leur occurence. Pour l'application d'un moteur de recherche, il vaut mieux donc utiliser le stemming, qui permet de rechercher les mots qui ont la même racine.

Temps d'éxécution de la méthode
(dépend du mot recherché, plus il est présent, plus la fonction sera longue)

In [59]:
d=time()
RankingOrderOcc(monIndexMap,'merchandise')
f=time()
print("temps d\'execution de RankingOrderOcc() : ", f-d)

temps d'execution de RankingOrderOcc() :  0.0005412101745605469



### 3.b. Ordonner les résultats d'une requête simple par fréquence (normalisée)

In [60]:
def RankingOrderFrq(index,word):
    articles=Research1Word(index,word) #utilisation de la fonction rechercher un mot
    #si le mot en paramètre n'existe pas, except de Research1Word
    wordBis=re.findall(r'[a-z]+', word.lower())
    dico={}
    for doc in articles: #pour chaque article
        #meme chose que precedemment en divisant par la taille du texte (doc[0]) pour obtenir la frequence
        dico[doc]=searchOcc(index,wordBis[0],doc)/searchNbMots(index,wordBis[0],doc)
    dicoSorted = sorted(dico.items(), reverse=True, key=lambda x:x[1]) #trier le dico
    return dicoSorted

Test de la fonction : 

In [61]:
RankingOrderFrq(monIndexMap,"merchandise")

[('RobinSidel/113551newsML.txt', 0.0136986301369863),
 ('JohnMastrini/159754newsML.txt', 0.00967741935483871),
 ('MartinWolk/183836newsML.txt', 0.009009009009009009),
 ('BradDorfman/331819newsML.txt', 0.006493506493506494),
 ('TheresePoletti/157001newsML.txt', 0.005037783375314861),
 ('JohnMastrini/123158newsML.txt', 0.004694835680751174),
 ('RobinSidel/220676newsML.txt', 0.004545454545454545),
 ('RobinSidel/219316newsML.txt', 0.004545454545454545),
 ('JohnMastrini/18297newsML.txt', 0.004484304932735426),
 ('JohnMastrini/18529newsML.txt', 0.004366812227074236),
 ('JohnMastrini/13806newsML.txt', 0.0035460992907801418),
 ('MarkBendeich/358701newsML.txt', 0.003367003367003367),
 ('BradDorfman/346831newsML.txt', 0.003367003367003367),
 ('MarkBendeich/358695newsML.txt', 0.0033222591362126247),
 ('BradDorfman/199931newsML.txt', 0.003003003003003003),
 ('PatriciaCommins/143591newsML.txt', 0.002976190476190476),
 ('MichaelConnor/372558newsML.txt', 0.002881844380403458),
 ('AaronPressman/156814

Temps d'éxécution de la méthode
(dépend du mot recherché, plus il est présent, plus la fonction sera longue)

In [62]:
d=time()
RankingOrderFrq(monIndexMap,'merchandise')
f=time()
print("temps d\'execution de RankingOrderFrq() : ", f-d)

temps d'execution de RankingOrderFrq() :  0.0011506080627441406


### 3.c. Ordonner les résultats d'une requête simple en utilisant la métrique TF-IDF

In [63]:
def RankingOrderTFIDF(index,word):
    articles=Research1Word(index,word)
    wordBis=re.findall(r'[a-z]+', word.lower())
    Nt=len(articles) #nombre de documents où apparait word
    D=len(listText) #nombre de documents dans l'index
    IDF=math.log(D / (1+Nt))
    TFIDF={}
    for doc in articles: #pour chaque article
        TFIDF[doc]=searchOcc(index,wordBis[0],doc)/searchNbMots(index,wordBis[0],doc)*IDF
    TFIDFsort =sorted(TFIDF.items(), reverse=True, key=lambda x:x[1])
    return TFIDFsort

Test de la fonction : 

In [64]:
RankingOrderTFIDF(monIndexMap,'merchandise')

[('RobinSidel/113551newsML.txt', 0.06364372850011434),
 ('JohnMastrini/159754newsML.txt', 0.04496121465008077),
 ('MartinWolk/183836newsML.txt', 0.0418557854099851),
 ('BradDorfman/331819newsML.txt', 0.030168780392911345),
 ('TheresePoletti/157001newsML.txt', 0.023405502168807792),
 ('JohnMastrini/123158newsML.txt', 0.021812169861541535),
 ('RobinSidel/220676newsML.txt', 0.021118146275037938),
 ('RobinSidel/219316newsML.txt', 0.021118146275037938),
 ('JohnMastrini/18297newsML.txt', 0.020834045652503796),
 ('JohnMastrini/18529newsML.txt', 0.020288175460735136),
 ('JohnMastrini/13806newsML.txt', 0.016475149576270734),
 ('MarkBendeich/358701newsML.txt', 0.015643071314842917),
 ('BradDorfman/346831newsML.txt', 0.015643071314842917),
 ('MarkBendeich/358695newsML.txt', 0.015435189968466267),
 ('BradDorfman/199931newsML.txt', 0.013951928469995035),
 ('PatriciaCommins/143591newsML.txt', 0.013827357680084364),
 ('MichaelConnor/372558newsML.txt', 0.013389026456796387),
 ('AaronPressman/156814new

Temps d'éxécution de la méthode
(dépend du mot recherché, plus il est présent, plus la fonction sera longue)

In [65]:
d=time()
RankingOrderTFIDF(monIndexMap,'merchandise')
f=time()
print("temps d\'execution de RankingOrderTFIDF() : ", f-d)

temps d'execution de RankingOrderTFIDF() :  0.0010395050048828125


### 3.d. Répondre à une requête complexe (=phrase) en implémentant le modèle d'espace vectoriel

Fonction pour rechercher un mot depuis un mot extrait de l'index (dans ce cas, pas besoin de faire les vérifications sur les stopWord ou stemming, puisque c'est déjà fait au moment de l'enregistrement du mot dans l'index)

In [68]:
def Research1WordInIndex(index,word):
    try:
        return set((index[word].keys())) #retourne les infos du mot
    except:
        print("Le mot " + word +" n'existe pas dans l'index \n")
        return set()

Fonction de création de la matrice selon la métrique choisie (par défaut, on prend TFIDF)

In [173]:
def createMat(index,metrique="TFIDF"):
    # Initialisation
    row = []
    col = []
    data = []
    listCorpus=createCorpusBis()
    D = len(listCorpus)  #ie. nombre d'articles dans le corpus
    nbIndex = len(index) # ie. nombre de mots dans l'index
    i=0
    # Pour tous les mots de l'index, on regarde tous les articles où il apparaît et on calcule le TF-IDF
    for word in index.keys():
        # indice i = ième mot de l'index
        Nt = len(Research1WordInIndex(index, word))
        IDF = math.log(D/(1+Nt))

        for doc in index[word].keys():
            
            col.append(i)
            row.append(index[word][doc]["idArticle"])
            if metrique=="BOW":
                data.append(1)
            else :
                if metrique=="nbOcc": #metrique choisie nombre d'occurence
                    data.append(index[word][doc]["occ"])
                else:
                    TF = index[word][doc]["occ"]/index[word][doc]["nbMots"]
                    if metrique=="frq": #metrique frequence
                        data.append(TF)
                    else: #par defaut on prend TF-IDF
                        data.append(TF*IDF)
        # On incrémente l'indice i pour chaque mot
        i = i + 1

    return coo_matrix((data,(row, col)),shape=(D,nbIndex)).toarray()

Temps d'éxécution de la méthode
(dépend du mot recherché, plus il est présent, plus la fonction sera longue)

In [174]:
d = time()
matriceBOW = createMat(monIndexMap,"BOW")
f = time()
print("Temps de création de la matrice avec le bag of word : \n", f-d)


d = time()
matriceNbOcc = createMat(monIndexMap,"nbOcc")
f = time()
print("Temps de création de la matrice avec le nombre d'occurences : \n", f-d)

d = time()
matriceFrq = createMat(monIndexMap,"frq")
f = time()
print("Temps de création de la matrice avec la fréquence : \n", f-d)

d = time()
matriceTFIDF = createMat(monIndexMap,"TFIDF")
f = time()
print("Temps de création de la matrice avec la métrique TF-IDF : \n", f-d)

Temps de création de la matrice avec le bag of word : 
 1.456125020980835
Temps de création de la matrice avec le nombre d'occurences : 
 0.405087947845459
Temps de création de la matrice avec la fréquence : 
 0.4499509334564209
Temps de création de la matrice avec la métrique TF-IDF : 
 0.42383337020874023


Affichage des matrices

In [175]:
print("Voici la matrice BOW : \n",matriceBOW)
print("\n")
print("Voici la matrice nbOcc : \n",matriceNbOcc)
print("\n")
print("Voici la matrice frq : \n",matriceFrq)
print("\n")
print("Voici la matrice TF-IDF : \n",matriceTFIDF)

Voici la matrice BOW : 
 [[1 1 1 ..., 0 0 0]
 [1 0 1 ..., 0 0 0]
 [1 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 1 1]
 [0 1 0 ..., 0 0 0]]


Voici la matrice nbOcc : 
 [[5 9 2 ..., 0 0 0]
 [1 0 1 ..., 0 0 0]
 [1 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 3 2 2]
 [0 1 0 ..., 0 0 0]]


Voici la matrice frq : 
 [[ 0.01453488  0.02616279  0.00581395 ...,  0.          0.          0.        ]
 [ 0.00269542  0.          0.00269542 ...,  0.          0.          0.        ]
 [ 0.003003    0.          0.003003   ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.02884615  0.01923077
   0.01923077]
 [ 0.          0.00398406  0.         ...,  0.          0.          0.        ]]


Voici la matrice TF-IDF : 
 [[ 0.02535048  0.06942459  0.00790382 ...,  0.          0.          0.        ]
 [ 0.00470111  0.          0.00366431 ...,  0.          0.     

Fonction qui recherche une phrase à l'aide de la matrice de choix (en texte libre)

In [72]:
def findWithMatrice(index, matrice, phrase):
    
    # Enlève les caractères spéciaux + minuscules + split
    phraseBis = re.findall(r'[a-z]+',phrase.lower())
    listePhrase = [word for word in phraseBis if word not in stop]
    listePhrase = [ps.stem(word) for word in listePhrase] ### STEMMING

    # Création d'une liste de taille de l'index remplis que de 0
    mots = [0 for i in range(len(index))]

    # Liste des mots de l'index
    keys = [k for k in index]

    # On transforme les 0 de la liste en 1 pour tous les mots de la phrase
    for mot in listePhrase:
        if mot in keys:
            position = keys.index(mot)
            mots[position] = 1

    # Produit matriciel avec la matrice TF-IDF
    pos = np.dot(matrice,mots)

    # Transformation de la liste
    pos = enumerate(pos)
    pos = sorted(pos, key=operator.itemgetter(1), reverse=True)
    indexes = [e[0] for e in pos if e[1] != 0]

    # Liste des fichiers
    articles=[]
    listCorpus=createCorpusBis()
    for i in indexes:
        articles.append(listCorpus[i]["titre"])

    return articles

#### Comparaison des matrices en temps

On choisit une phrase : 

In [210]:
phrase ="The MercHaNdise Coucou Trade DEficit"

Temps de recherche d'un phrase à partir de la matrice bag of word

In [211]:
d = time()
researchBOW=findWithMatrice(monIndexMap,matriceBOW,phrase)
f = time()
print("Temps de recherche de la phrase (BOW) : ", f-d)
tpsBOW=f-d

Temps de recherche de la phrase (BOW) :  0.0985708236694336


Temps de recherche d'un phrase à partir de la matrice nombre d'occurences

In [180]:
d = time()
researchNbOcc=findWithMatrice(monIndexMap,matriceNbOcc,phrase)
f = time()
print("Temps de recherche de la phrase (NbOcc) : ", f-d)
tpsNbOcc=f-d

Temps de recherche de la phrase (NbOcc) :  0.10541939735412598


Temps de recherche d'un phrase à partir de la matrice fréquence

In [181]:
d = time()
researchFrq=findWithMatrice(monIndexMap,matriceFrq,phrase)
f = time()
print("Temps de recherche de la phrase (Frq) : ", f-d)
tpsFrq=f-d

Temps de recherche de la phrase (Frq) :  0.5546603202819824


Temps de recherche d'un phrase à partir de la matrice métrique TF-IDF

In [182]:
d = time()
researchTFIDF=findWithMatrice(monIndexMap,matriceTFIDF,phrase)
f = time()
print("Temps de recherche de la phrase (TFIDF) : ", f-d)
tpsTFIDF=f-d

Temps de recherche de la phrase (TFIDF) :  0.08728647232055664


Voici les temps de recherches triés par ordre croissant : 

In [187]:
listTps = [("BOW",tpsBOW),("nbOcc", tpsNbOcc), ("frq", tpsFrq), ("TFIDF", tpsTFIDF)]
listTpsSort = sorted(listTps, key=lambda colonne:colonne[1]) #trier le dico pour ordonner resultat
listTpsSort

[('TFIDF', 0.08728647232055664),
 ('BOW', 0.0967707633972168),
 ('nbOcc', 0.10541939735412598),
 ('frq', 0.5546603202819824)]

--> Le temps le plus court est TFIDF

#### Comparaison des matrices avec les sorties

Affichage de la recherche avec bag of word

In [188]:
print("résultats avec matrice NbOBOWcc : \n", researchBOW)

résultats avec matrice NbOBOWcc : 
 ['JohnMastrini/159754newsML.txt', 'JohnMastrini/18297newsML.txt', 'JohnMastrini/18529newsML.txt', 'JohnMastrini/13806newsML.txt', 'JohnMastrini/123158newsML.txt', 'SarahDavison/386644newsML.txt', 'SarahDavison/365768newsML.txt', 'SarahDavison/213002newsML.txt', 'JonathanBirt/327653newsML.txt', 'AaronPressman/156814newsML.txt', 'EricAuchard/149671newsML.txt', 'ScottHillis/299129newsML.txt', 'WilliamKazer/208244newsML.txt', 'WilliamKazer/178884newsML.txt', 'WilliamKazer/180188newsML.txt', 'AlanCrosby/139110newsML.txt', 'AlanCrosby/135777newsML.txt', 'AlanCrosby/140051newsML.txt', 'AlanCrosby/136471newsML.txt', 'AlanCrosby/216602newsML.txt', 'AlanCrosby/217025newsML.txt', 'DarrenSchuettler/144792newsML.txt', 'DarrenSchuettler/168489newsML.txt', 'DarrenSchuettler/383585newsML.txt', 'DarrenSchuettler/144056newsML.txt', 'DarrenSchuettler/174369newsML.txt', 'LydiaZajc/130081newsML.txt', 'KeithWeir/275481newsML.txt', 'RobinSidel/113551newsML.txt', 'JanLopatk

Affichage de la recherche avec nombre d'occurences

In [86]:
print("résultats avec matrice NbOcc : \n", researchNbOcc)

résultats avec matrice NbOcc : 
 ['WilliamKazer/208244newsML.txt', 'JohnMastrini/159754newsML.txt', 'ScottHillis/299129newsML.txt', 'JohnMastrini/105045newsML.txt', 'JanLopatka/163409newsML.txt', 'JanLopatka/163094newsML.txt', 'MureDickie/187394newsML.txt', 'AaronPressman/401260newsML.txt', 'KevinDrawbaugh/191657newsML.txt', 'WilliamKazer/235763newsML.txt', 'AlanCrosby/135777newsML.txt', 'AlanCrosby/136471newsML.txt', 'GrahamEarnshaw/172788newsML.txt', 'JohnMastrini/13806newsML.txt', 'AlanCrosby/216602newsML.txt', 'AlanCrosby/217025newsML.txt', 'AaronPressman/182596newsML.txt', 'MureDickie/186174newsML.txt', 'JaneMacartney/246277newsML.txt', 'JaneMacartney/247009newsML.txt', 'ScottHillis/140340newsML.txt', 'ScottHillis/142465newsML.txt', 'FumikoFujisaki/10028newsML.txt', 'FumikoFujisaki/23340newsML.txt', 'JohnMastrini/18297newsML.txt', 'JohnMastrini/18529newsML.txt', "LynneO'Donnell/169600newsML.txt", 'SarahDavison/396743newsML.txt', 'AlanCrosby/139110newsML.txt', 'GrahamEarnshaw/27116

Affichage de la recherche avec fréquences

In [87]:
print("résultats avec matrice Frq : \n", researchFrq)


résultats avec matrice Frq : 
 ['AlanCrosby/136471newsML.txt', 'KevinDrawbaugh/191657newsML.txt', 'WilliamKazer/208244newsML.txt', 'AlanCrosby/135777newsML.txt', 'ScottHillis/299129newsML.txt', 'JohnMastrini/159754newsML.txt', 'JanLopatka/163409newsML.txt', 'JanLopatka/163094newsML.txt', 'GrahamEarnshaw/172788newsML.txt', 'MureDickie/187394newsML.txt', 'AlanCrosby/216602newsML.txt', 'AlanCrosby/217025newsML.txt', 'JohnMastrini/13806newsML.txt', 'AlanCrosby/139110newsML.txt', 'JohnMastrini/18297newsML.txt', 'WilliamKazer/235763newsML.txt', 'JohnMastrini/18529newsML.txt', 'AlanCrosby/140051newsML.txt', 'LydiaZajc/185720newsML.txt', 'MureDickie/186174newsML.txt', 'AaronPressman/401260newsML.txt', 'ScottHillis/140340newsML.txt', 'ScottHillis/142465newsML.txt', 'WilliamKazer/147523newsML.txt', 'WilliamKazer/178884newsML.txt', 'WilliamKazer/180188newsML.txt', "LynneO'Donnell/169600newsML.txt", 'JohnMastrini/105045newsML.txt', 'FumikoFujisaki/10028newsML.txt', 'JaneMacartney/253835newsML.txt'

Affichage de la recherche avec TF-IDF

In [88]:

print("résultats avec matrice TFIDF : \n", researchTFIDF)

résultats avec matrice TFIDF : 
 ['AlanCrosby/136471newsML.txt', 'AlanCrosby/135777newsML.txt', 'JohnMastrini/159754newsML.txt', 'JohnMastrini/13806newsML.txt', 'JohnMastrini/18297newsML.txt', 'JanLopatka/163409newsML.txt', 'JohnMastrini/18529newsML.txt', 'JanLopatka/163094newsML.txt', 'WilliamKazer/208244newsML.txt', 'AlanCrosby/140051newsML.txt', 'ScottHillis/299129newsML.txt', 'AlanCrosby/139110newsML.txt', 'AlanCrosby/216602newsML.txt', 'AlanCrosby/217025newsML.txt', 'JohnMastrini/105045newsML.txt', 'KevinDrawbaugh/191657newsML.txt', 'JohnMastrini/123158newsML.txt', 'RobinSidel/113551newsML.txt', 'SarahDavison/213002newsML.txt', 'DarrenSchuettler/383585newsML.txt', 'GrahamEarnshaw/172788newsML.txt', 'MureDickie/187394newsML.txt', 'JohnMastrini/266394newsML.txt', 'WilliamKazer/235763newsML.txt', 'WilliamKazer/178884newsML.txt', 'WilliamKazer/180188newsML.txt', 'JanLopatka/10224newsML.txt', 'LydiaZajc/185720newsML.txt', 'MureDickie/186174newsML.txt', 'KarlPenhaul/305637newsML.txt', '

On remarque que les articles en sortie sont différents selon la méthode. Le premier article avec le nombre d'occurence n'est pas forcément la même qu'avec la fréquence ou la métrique TF-IDF.

Par exemple, pour la phrase choisie, l'article "AlanCrosby/135777newsML.txt" est en 2e position pour TFIDF. Pour la fréquence, l'article est 4e. Et pour le nombre d'occurences, l'article est en 11e. et 17e pour Bag of Word

Affichage recherche matrice

In [222]:
def PrintResearchMatrice(index):
    cont="oui"
    wordUse=[]
    while(cont=="oui"):
        phrasePrint=input("Indiquez la phrase que vous souhaitez rechercher : ")
        print("\n")
        metriquePrint=input("Indiquez la matrice que vous souhaitez utiliser ('BOW', 'nbOcc', 'frq', 'TFIDF': ")
        print("\n")
        matriceUse=createMat(monIndexMap,metriquePrint)
        listDoc=findWithMatrice(index,matriceUse, phrasePrint)
        print("nombre de documents = %d \n" %len(listDoc))
        if len(listDoc)>0:
            print("Voici les 10 premiers éléments : \n ", set(itertools.islice(listDoc,10)))
        cont=input("Souhaitez vous rechercher une autre phrase ? (oui/non) \n").lower()
    print("Fin de recherche en phrase")

In [223]:
PrintResearchMatrice(monIndexMap)

Indiquez la phrase que vous souhaitez rechercher : The MercHaNdise Coucou Trade DEficit


Indiquez la matrice que vous souhaitez utiliser ('BOW', 'nbOcc', 'frq', 'TFIDF': BOW


nombre de documents = 886 

Voici les 10 premiers éléments : 
  {'SarahDavison/386644newsML.txt', 'SarahDavison/365768newsML.txt', 'JohnMastrini/18529newsML.txt', 'JohnMastrini/159754newsML.txt', 'JohnMastrini/13806newsML.txt', 'JohnMastrini/123158newsML.txt', 'AaronPressman/156814newsML.txt', 'JohnMastrini/18297newsML.txt', 'JonathanBirt/327653newsML.txt', 'SarahDavison/213002newsML.txt'}
Souhaitez vous rechercher une autre phrase ? (oui/non) 
non
Fin de recherche en phrase


# Piste d'amélioration : Lemmatization
Utiliser la lemmatization pour pallier aux problèmes de stemming (verbe, gestion des pays)

BUT : créer un nouvel index, et réadapter toutes les méthodes, puis comparer les sorties avec l'index Stemming

Les imports nécessaires pour la lemmatization

In [229]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Exemple d'utilisation de la lemmatization

In [226]:
exPhrase="The student is studying for his exams very consciously "
text = re.findall(r'[a-z]+', exPhrase.lower()) ##NORMALISATION
text = [word for word in text if word not in stop] #STOPWORDS
s=' '.join(text) #transformation d'une liste en phrase
tokens = word_tokenize(s) # generer les tokens

tokens_pos = pos_tag(tokens)  #generer les positions des tokens

print(tokens_pos)


[('student', 'NN'), ('studying', 'VBG'), ('exams', 'NNS'), ('consciously', 'RB')]


Dictionnaire qui modifie les valeurs des pos_tag

In [227]:
dicPos= {'JJ': wn.ADJ,'JJR': wn.ADJ,'JJS': wn.ADJ,'RB':wn.ADV,'RBR': wn.ADV,'RBS': wn.ADV,'NN': wn.NOUN,'NNP': wn.NOUN,
    'NNS': wn.NOUN,'NNPS': wn.NOUN,'VB': wn.VERB,'VBG':  wn.VERB,'VBD':  wn.VERB, 'VBN':  wn.VERB,'VBP':  wn.VERB,'VBZ':  wn.VERB
}

lemmatization sur la phrase

In [228]:
for word in tokens_pos:
    print("Lemmatise : %s" % (wnl.lemmatize(word[0], pos=dicPos[word[1]])))

Lemmatise : student
Lemmatise : study
Lemmatise : exam
Lemmatise : consciously


Différence avec le stemming

In [193]:
for word in tokens:
    print("Stemming: %s" % (ps.stem(word)))

Stemming: student
Stemming: studi
Stemming: exam
Stemming: conscious


## Indexation avec Lemmatization

In [194]:
def mapIndexLem(article):
    text = re.findall(r'[a-z]+', article["texte"].lower())
    text = [word for word in text if word not in stop]
    ####LEMMATIZATION
    s=' '.join(text) #transformation d'une liste en phrase
    tokens = word_tokenize(s) # generer les tokens
    tokens_pos = pos_tag(tokens)  #generer les positions des tokens
    text = [wnl.lemmatize(word[0], pos=dicPos[word[1]]) for word in tokens_pos if(word[1] in dicPos)]
    size = len(text) #taille du texte


    indexRes = {} #initialisation index de sortie (dictionnaire)
    #parcourir chaque mot du texte i
    for i in range(0, len(text)) :
        word=text[i]
        #si le mot n'est pas déjà présent dans l'index
        if (word not in indexRes.keys()) :
            indexRes[word]={} ## creation du dictionnaire
            indexRes[word][article["titre"]]={} ##creation du second dictionnaire avec pour clé le titre (auteur/article)
            indexRes[word][article["titre"]]["pos"]=[]# initialisation de la valeur position
            indexRes[word][article["titre"]]["nbMots"]=size #indique le nbMots dans le texte
            indexRes[word][article["titre"]]["idArticle"]=article["id"]
        indexRes[word][article["titre"]]["pos"].append(i) #ajoute la nouvelle position
    
    #boucle pour pour valoriser l'occurence du mot
    for word in indexRes.keys():
        indexRes[word][article["titre"]]["occ"]=len( indexRes[word][article["titre"]]["pos"])
    return indexRes

In [195]:
 def createIndexMapRedLem():
    listText=createCorpusBis() #creation du corpus (liste de dictionnaire)
    with Pool() as pool:
        #Map
        dMap = time()
        listIndex=pool.map(mapIndexLem,listText) #appel fonction map sur listText
        fMap = time()
        print('temps d\'execution du Map : ', fMap-dMap)
        
        #Reduce
        dRed = time()
        indexRes=reduce(reduceIndex,listIndex) #reduce sur listIndex en sortie du map
        fRed = time()
        print('temps d\'execution du Reduce : ', fRed-dRed)
    return indexRes

In [196]:
d = time()
monIndexMapLem=createIndexMapRedLem()
f = time()
print('temps d\'execution total de createIndexMapReduceLem', f-d)

temps d'execution du Map :  17.90281367301941
temps d'execution du Reduce :  0.4356975555419922
temps d'execution total de createIndexMapReduceLem 19.15755295753479


In [168]:
print("Nombre de mots différents dans l'index avec stemming : ", len(monIndexMap))
print("Nombre de mots différents dans l'index avec lemmatization : ", len(monIndexMapLem))
print("Nombre de mots différents dans l'index : ", len(monIndex))


Nombre de mots différents dans l'index avec stemming :  18456
Nombre de mots différents dans l'index avec lemmatization :  22583
Nombre de mots différents dans l'index :  27361


In [170]:
dict(itertools.islice(monIndexMapLem["merchandise"].items(), 10))

{'AaronPressman/156814newsML.txt': {'idArticle': 305,
  'nbMots': 341,
  'occ': 1,
  'pos': [50]},
 'BradDorfman/199931newsML.txt': {'idArticle': 1207,
  'nbMots': 321,
  'occ': 1,
  'pos': [34]},
 'BradDorfman/276360newsML.txt': {'idArticle': 1217,
  'nbMots': 405,
  'occ': 1,
  'pos': [208]},
 'BradDorfman/331819newsML.txt': {'idArticle': 1214,
  'nbMots': 290,
  'occ': 1,
  'pos': [187]},
 'BradDorfman/346831newsML.txt': {'idArticle': 1222,
  'nbMots': 273,
  'occ': 1,
  'pos': [64]},
 'KeithWeir/275481newsML.txt': {'idArticle': 1639,
  'nbMots': 500,
  'occ': 1,
  'pos': [103]},
 'MichaelConnor/372558newsML.txt': {'idArticle': 1144,
  'nbMots': 333,
  'occ': 1,
  'pos': [188]},
 'RobinSidel/113551newsML.txt': {'idArticle': 1743,
  'nbMots': 281,
  'occ': 3,
  'pos': [141, 202, 209]},
 'RobinSidel/219316newsML.txt': {'idArticle': 1701,
  'nbMots': 210,
  'occ': 1,
  'pos': [188]},
 'RobinSidel/220676newsML.txt': {'idArticle': 1736,
  'nbMots': 211,
  'occ': 1,
  'pos': [189]}}

In [225]:
dict(itertools.islice(monIndexMapLem["merchandiser"].items(), 10))

{'PatriciaCommins/143591newsML.txt': {'idArticle': 536,
  'nbMots': 321,
  'occ': 1,
  'pos': [188]},
 'RobinSidel/113551newsML.txt': {'idArticle': 1743,
  'nbMots': 281,
  'occ': 1,
  'pos': [187]}}

### Adapter les méthodes de requetes et de ranking

# DEMONSTRATION

Recheche à un mot : 

In [ ]:
PrintResearch1(monIndexMap)

Recheche d'un texte libre : 
- MercHANdisers traDer

In [ ]:
PrintResearch(monIndexMap)

Recheche d'une phrase :
- The MercHaNdise Trade DEficit
- Lloyd's of London Reconstruction and Renewal plan
- day


In [ ]:
PrintResearchSeq(monIndexMap)

## Avec les matrices

In [ ]:
PrintResearchMatrice(monIndexMap)